# Capstone Project Notebook - Week 3 - Toronton clustering

## Step 1 : Get data from Wikipedia webpage

In [166]:
#Import Library
import requests
import pandas as pd
import wikipedia as wp

In [167]:
# Ignore Unverified HTTPS request
import os
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
os.environ['CURL_CA_BUNDLE'] = ""
os.environ['PYTHONWARNINGS']="ignore:Unverified HTTPS request"

In [168]:
#Get the html source in Wikipedia
html = wp.page("List_of_postal_codes_of_Canada: M").html().encode("UTF-8")

In [169]:
# Read with pandapd.DataFrame(
df_wiki = pd.read_html(html)
df_wiki = pd.DataFrame(df_wiki[0])
df_wiki.head()

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park / Ontario Provincial Government,M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


In [170]:
#Initialize the final dataframe
df_toronto = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])

In [171]:
# Fill in the final dataframe

# Loop over the lines
for ii in range(df_wiki.shape[0]):
    # Loop over the columns
    for jj in range(df_wiki.shape[1]):
        
        # Cell content
        cell=df_wiki.iloc[ii,jj]
        
        # Postcode
        postcode=cell[0:3]
        
        # Remaining info
        name=cell[3:]
        
        # Ignore Not assigned values
        if name != 'Not assigned':
            
            #print(postcode)
            split_name = name.split('(')
            
            # Borough list
            borough_list = split_name[0].split('/')
            #print(borough_list)
            
            # Neighbourhood list
            if len(split_name)>1:
                neighbourhood_list = name.split('(')[1].split('/')
                neighbourhood_list[-1] = neighbourhood_list[-1].split(')')[0]
            else:
                neighbourhood_list=borough_list
            #print(neighbourhood_list)    
            
            for kk in range(len(borough_list)):
                
                borough = borough_list[kk]
                
                for ll in range(len(neighbourhood_list)):
                        
                    neighbourhood = neighbourhood_list[ll]
                            
                    # Add the row to the dataframe
                    df_toronto.loc[len(df_toronto)]=[postcode,borough,neighbourhood]

df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Manor


In [172]:
# Group by
df_toronto = pd.DataFrame(df_toronto.groupby(by=['PostalCode','Borough'])['Neighborhood']\
                          .apply(lambda x: "%s" % ', '.join(x)))

In [173]:
#Reset index
df_toronto= df_toronto.reset_index()
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [174]:
df_toronto.shape

(104, 3)